# shape-detection

In [1]:
import cv2
import numpy as np

# Load the images
image1 = cv2.imread('shape1.jpg')
image2 = cv2.imread('shape.jpg')
width, height = 500, 500
# Ensure all images have the same dimensions
# For simplicity, let's resize all images to the size of the first image
image1 = cv2.resize(image1, (width, height))
image2 = cv2.resize(image2, (width, height))
# Concatenate the images horizontally
concatenated_image = np.hstack((image1, image2))

# Display the concatenated image
cv2.imshow('Concatenated Images', concatenated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
import cv2
import numpy as np

def detect_shapes(image_path):
    # Read the image
    image = cv2.imread(image_path)
    
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply GaussianBlur to reduce noise
    blurred = cv2.GaussianBlur(gray, (7, 7), 0)
    
    # Perform edge detection using Canny
    edges = cv2.Canny(blurred, 30, 150)
    #edges = cv2.threshold(blurred, 220, 255, cv2.THRESH_BINARY_INV)[1]
    #cv2.imshow("dd", edges)
    #cv2.waitKey(0)
    
    contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Loop over the contours
    for contour in contours:
        # Approximate the contour
        epsilon = 0.01 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        
        # Determine the shape based on the number of vertices
        num_vertices = len(approx)
        shape = "unknown"
        
        if num_vertices == 3:
            shape = "triangle"
        elif num_vertices == 4:
            # Check if it's a square or rectangle
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = float(w) / h
            shape = "square" if 0.99 <= aspect_ratio <= 1.01 else "rectangle"
        elif num_vertices == 5:
            shape = "pentagon"
        elif cv2.arcLength(contour, True) > 0:
            # Calculate the center and radius of the contour to determine if it's a circle
            (x, y), radius = cv2.minEnclosingCircle(contour)
            area = cv2.contourArea(contour)
            circularity = 4 * np.pi * area / (cv2.arcLength(contour, True) ** 2)
            shape = "circle" if circularity >= 0.85 else "oval"
        else:
            shape = "unknown"
        
        # Draw the contour and the shape name on the image
        cv2.drawContours(image, [approx], -1, (0, 255, 0), 3)
        cv2.putText(image, shape, (approx.ravel()[0], approx.ravel()[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
    
    # Display the image with detected shapes
    cv2.imshow("Shapes Detection", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Path to the image file
image_path = 'shape1.jpg'
#image_path = 'shape.jpg'
# Call the function to detect shapes
detect_shapes(image_path)
